# Balancing Data — ENTSO-E Examples

Querying imbalance prices and volumes.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from entsoe import Client

client = Client()

## 1. Imbalance Prices — France (1 week)

In [ ]:
start = "2024-06-01"
end = "2024-06-08"

df_prices = client.balancing.imbalance_prices(start, end, country="FR")
df_prices.head()

In [ ]:
fig = px.line(
    df_prices, x="timestamp", y="value",
    title="Imbalance Prices — France",
    labels={"value": "Price (EUR/MWh)", "timestamp": ""},
)
fig.show()

## 2. Imbalance Prices — Multi-Country

In [ ]:
countries = {"FR": "France", "NL": "Netherlands", "BE": "Belgium"}
frames = []
for code, name in countries.items():
    df = client.balancing.imbalance_prices(start, end, country=code)
    df["country"] = name
    frames.append(df)

df_multi = pd.concat(frames, ignore_index=True)

fig = px.line(
    df_multi, x="timestamp", y="value", color="country",
    title="Imbalance Prices — Multi-Country Comparison",
    labels={"value": "Price (EUR/MWh)", "timestamp": ""},
)
fig.show()

## 3. Price Distribution — Histogram

In [ ]:
fig = px.histogram(
    df_multi, x="value", color="country", barmode="overlay",
    nbins=50, opacity=0.7,
    title="Imbalance Price Distribution",
    labels={"value": "Price (EUR/MWh)"},
)
fig.show()

## 4. Imbalance Prices vs Day-Ahead Prices

In [ ]:
df_da = client.prices.day_ahead(start, end, country="FR")
df_imb = client.balancing.imbalance_prices(start, end, country="FR")

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_da["timestamp"], y=df_da["value"],
    name="Day-Ahead", line=dict(color="#636EFA"),
))
fig.add_trace(go.Scatter(
    x=df_imb["timestamp"], y=df_imb["value"],
    name="Imbalance", line=dict(color="#EF553B"),
))
fig.update_layout(
    title="Day-Ahead vs Imbalance Prices — France",
    yaxis_title="Price (EUR/MWh)",
    xaxis_title="",
)
fig.show()